In [4]:
import requests
from bs4 import BeautifulSoup

# Exercise 1

- Initialize

In [130]:
url_KV = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'
r = requests.get(url)
soup = BeautifulSoup(r.content,'html.parser')

# Contains values for :
# ww_x_UNITE_ACAD
# ww_x_PERIODE_ACAD
# ww_x_PERIODE_PEDAGO
# ww_x_HIVERETE
table = soup.find(id='filtre')
unite_acad = table.find('select', {'name':'ww_x_UNITE_ACAD'})
info = unite_acad.find('option', text='Informatique')
print("Key 'Informatique' has value: "+info['value'])


Key 'Informatique' has value: 249847


# Exercise 2